In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [2]:
sales = pd.read_csv('data/2022-04-01T12_df_sales.csv')
web_logs = pd.read_csv('data/2022-04-01T12_df_web_logs.csv')
users = pd.read_csv('data/experiment_users.csv')

In [3]:
sales["date"] = pd.to_datetime(sales["date"])
web_logs["date"]  = pd.to_datetime(web_logs["date"])

Эксперимент проводился с 03.23.2022 по 30.03.2022 (7 дней)

## Исторические значения и А/А тест

In [4]:
from useful_functions import get_revenue_all, get_revenue_web, get_response_time
from useful_functions import aa_pvalues, plot_pvalue_ecdf_hist, plot_pvalue_ecdf_prob
begin_date = pd.to_datetime("16.03.2022", dayfirst=True).date()
end_date   = pd.to_datetime("23.03.2022", dayfirst=True).date()
main_metric = get_revenue_all(sales, web_logs, begin_date, end_date)
assist_metric = get_revenue_web(sales, web_logs, begin_date, end_date)
control_metric = get_response_time(web_logs, begin_date, end_date).groupby('user_id')[['load_time']].mean().reset_index()

m = main_metric.set_index("user_id")["total_revenue"]

pvals_main = aa_pvalues(m)
plot_pvalue_ecdf_hist(pvals_main, "A/A p-values — revenue_all (hist)")
plot_pvalue_ecdf_prob(pvals_main, "A/A p-values — revenue_all (ECDF)")

In [5]:
a = assist_metric.set_index("user_id")["total_revenue_web"]

pvals_assist = aa_pvalues(a)
plot_pvalue_ecdf_hist(pvals_assist, "A/A p-values — revenue_web (hist)")
plot_pvalue_ecdf_prob(pvals_assist, "A/A p-values — revenue_web (ECDF)")

In [6]:
c = control_metric.set_index("user_id")["load_time"]

pvals_control = aa_pvalues(c)
plot_pvalue_ecdf_hist(pvals_control, "A/A p-values — response_time (hist)")
plot_pvalue_ecdf_prob(pvals_control, "A/A p-values — response_time (ECDF)")

In [7]:
from useful_functions import describe_metric, plot_metric_hist

print(describe_metric(m, "revenue_all"))
plot_metric_hist(m, "revenue_all distribution (baseline week)", log1p=True)

        metric  n_users        mean         std  median  p75    p90     p95  \
0  revenue_all   115278  263.795347  632.731435     0.0  0.0  840.0  1560.0   

      p99  share_zero  
0  2940.0    0.786603  


In [8]:
print(describe_metric(a, "revenue_web"))
plot_metric_hist(a, "total web revenue distribution (baseline week)", log1p=True)

        metric  n_users        mean         std  median     p75     p90  \
0  revenue_web    34777  874.422751  890.579974   720.0  1320.0  2070.0   

      p95     p99  share_zero  
0  2580.0  4050.0    0.292636  


In [9]:
print(describe_metric(c, "load_time"))
plot_metric_hist(c, "average load_time distribution (baseline week)", log1p=False)

      metric  n_users       mean        std     median    p75    p90     p95  \
0  load_time    34777  74.234867  49.071007  69.914286  73.25  76.55  79.128   

        p99  share_zero  
0  255.7215    0.000029  


In [10]:
from useful_functions import get_mde
alpha = 0.05
beta_target = 0.2
n_per_group = 57639

std_log = np.log1p(m).std(ddof=1)
mde_log = get_mde(std_log, n_per_group, alpha=alpha, beta=beta_target, two_sided=True)

print("std(log1p):", std_log)
print("MDE(log1p):", mde_log, "≈", (np.exp(mde_log) - 1), "relative (rough)")



std(log1p): 2.8627249690528185
MDE(log1p): 0.04724330178460868 ≈ 0.04837705005965698 relative (rough)


In [11]:
from useful_functions import run_experiment_pvalues
pvals_aa, alpha_hat = run_experiment_pvalues(
    baseline=m,
    n_per_group=n_per_group,
    n_iter=2000,
    alpha=alpha,
    effect=0.0,         
    transform="log1p",
    test="ttest",
    seed=1
)

In [12]:
pvals_ab, power_hat = run_experiment_pvalues(
    baseline=m,
    n_per_group=n_per_group,
    n_iter=2000,
    alpha=alpha,
    effect=mde_log,      
    transform="log1p",
    test="ttest",
    seed=2
)

beta_hat = 1 - power_hat
print(f"Empirical alpha (Type I): {alpha_hat:.3f}")
print(f"Empirical beta  (Type II): {beta_hat:.3f}")
print(f"Empirical power: {power_hat:.3f}")

Empirical alpha (Type I): 0.053
Empirical beta  (Type II): 0.205
Empirical power: 0.795


### Выводы

Ошибка I рода (α ≈ 0.053)

Ожидалось: α = 0.05

Получили: 0.053 - тест не переотклоняет нулевую гипотезу.

Реализация теста и выбранная статистика адекватны

Ошибка II рода (β ≈ 0.205)

Целевое значение: β = 0.2

Получили: 0.205 - при эффекте, равном рассчитанному MDE, вероятность не обнаружить эффект составляет около 20%, что соответствует ожиданиям. Практически полное совпадение.

Мощность теста (power ≈ 0.795)

Целевая мощность: 0.8

Эмпирическая: 0.795 - при размере групп 57639 пользователей тест способен обнаруживать минимальный эффект порядка ~4.8% с вероятностью около 80%. Но в нашем распоряжении будет не так много пользователей, поэтому мы сократим их количество для проверки гипотез о веб-приложении.

In [13]:
alpha = 0.05
beta_target = 0.2
n_per_group = 17388

std_log = np.log1p(a).std(ddof=1)
mde_log = get_mde(std_log, n_per_group, alpha=alpha, beta=beta_target, two_sided=True)

print("std(log1p):", std_log)
print("MDE(log1p):", mde_log, "≈", (np.exp(mde_log) - 1), "relative (rough)")

std(log1p): 3.198622593112559
MDE(log1p): 0.09610744237060287 ≈ 0.10087733851101044 relative (rough)


In [14]:
pvals_aa, alpha_hat = run_experiment_pvalues(
    baseline=a,
    n_per_group=n_per_group,
    n_iter=2000,
    alpha=alpha,
    effect=0.0,         
    transform="log1p",
    test="ttest",
    seed=1
)
pvals_ab, power_hat = run_experiment_pvalues(
    baseline=a,
    n_per_group=n_per_group,
    n_iter=2000,
    alpha=alpha,
    effect=mde_log,      
    transform="log1p",
    test="ttest",
    seed=2
)

beta_hat = 1 - power_hat
print(f"Empirical alpha (Type I): {alpha_hat:.3f}")
print(f"Empirical beta  (Type II): {beta_hat:.3f}")
print(f"Empirical power: {power_hat:.3f}")

Empirical alpha (Type I): 0.057
Empirical beta  (Type II): 0.192
Empirical power: 0.808


При размере групп примерно 17 тыс. пользователей тест способен детектить эффекты порядка ~10% с вероятностью около 80%. Тест в целом корректно контролирует уровень значимости и не склонен к систематическому переотклонению. А значит мы можем переходить к А/Б тестированию.

## A/B тестирование

In [4]:
from datetime import datetime
from useful_functions import build_experiment_metrics, ttest_pvalue

exp_begin = pd.to_datetime("23.03.2022", dayfirst=True).date()
exp_end = pd.to_datetime("30.03.2022", dayfirst=True).date()

df_exp = build_experiment_metrics(users, sales, web_logs, exp_begin, exp_end)

metrics = ["total_revenue", "total_revenue_web", "orders_cnt"]

for col in metrics:
    p = ttest_pvalue(df_exp[df_exp["pilot"] == 0][col], df_exp[df_exp["pilot"] == 1][col])
    print(f"{col}: t-test pvalue = {p:.4g}")

p_rt = ttest_pvalue(
    df_exp[df_exp["pilot"] == 0]["avg_load_time"],
    df_exp[df_exp["pilot"] == 1]["avg_load_time"],
)
print(f"avg_load_time: t-test pvalue = {p_rt:.4g}")


total_revenue: t-test pvalue = 6.505e-15
total_revenue_web: t-test pvalue = 6.505e-15
orders_cnt: t-test pvalue = 7.675e-27
avg_load_time: t-test pvalue = 0.1544


In [5]:
from useful_functions import build_covariates_for_users, cuped_pvalue
df_cov = build_covariates_for_users(users, sales, web_logs, exp_begin, exp_end, cov_days=(7, 28))

df_for_cuped = df_exp.merge(df_cov.drop(columns=["pilot"]), on="user_id", how="left")

p_plain = ttest_pvalue(
    df_for_cuped[df_for_cuped["pilot"] == 0]["total_revenue"],
    df_for_cuped[df_for_cuped["pilot"] == 1]["total_revenue"],
)

p_cuped_7 = cuped_pvalue(df_for_cuped, metric_col="total_revenue", cov_col="cov_7d")
p_cuped_28 = cuped_pvalue(df_for_cuped, metric_col="total_revenue", cov_col="cov_28d")

print(f"revenue_all pvalue (plain t-test): {p_plain:.4g}")
print(f"revenue_all pvalue (CUPED cov_7d):  {p_cuped_7:.4g}")
print(f"revenue_all pvalue (CUPED cov_28d): {p_cuped_28:.4g}")

revenue_all pvalue (plain t-test): 6.505e-15
revenue_all pvalue (CUPED cov_7d):  1.011e-14
revenue_all pvalue (CUPED cov_28d): 1.282e-13


In [11]:
from useful_functions import calculate_theta

def cuped_theta_and_variance_reduction(df, metric_col, cov_col, group_col="pilot"):
    df2 = df[[group_col, metric_col, cov_col]].fillna(0)

    control = df2[df2[group_col] == 0]
    pilot   = df2[df2[group_col] == 1]

    theta = calculate_theta(
        control[metric_col].reset_index(drop=True),
        pilot[metric_col].reset_index(drop=True),
        control[cov_col].reset_index(drop=True),
        pilot[cov_col].reset_index(drop=True),
    )

    # CUPED-метрика
    m_cuped = df2[metric_col] - theta * df2[cov_col]

    var_raw = float(df2[metric_col].var(ddof=1))
    var_cup = float(m_cuped.var(ddof=1))

    # corr по всем (это то, что "объяснимость" даёт)
    corr = float(df2[[metric_col, cov_col]].corr().iloc[0, 1])

    reduction = 1 - (var_cup / var_raw) if var_raw > 0 else 0.0
    return theta, corr, var_raw, var_cup, reduction


for cov in ["cov_7d", "cov_28d"]:
    theta, corr, var_raw, var_cup, red = cuped_theta_and_variance_reduction(
        df_for_cuped, metric_col="total_revenue", cov_col=cov
    )
    print(f"\nCUPED diagnostics for {cov}:")
    print(f"  corr(metric, cov): {corr:.3f}")
    print(f"  theta:             {theta:.4f}")
    print(f"  var raw:           {var_raw:.3f}")
    print(f"  var CUPED:         {var_cup:.3f}")
    print(f"  variance reduction:{red*100:.1f}%")


CUPED diagnostics for cov_7d:
  corr(metric, cov): 0.052
  theta:             0.0913
  var raw:           783348.987
  var CUPED:         781268.602
  variance reduction:0.3%

CUPED diagnostics for cov_28d:
  corr(metric, cov): 0.459
  theta:             0.3034
  var raw:           783348.987
  var CUPED:         618645.727
  variance reduction:21.0%


In [12]:
import plotly.express as px

df_cmp = pd.DataFrame({
    "method": ["t-test", "CUPED cov_7d", "CUPED cov_28d"],
    "p_value": [p_plain, p_cuped_7, p_cuped_28]
})
fig = px.bar(df_cmp, x="method", y="p_value", title="Real experiment: p-values comparison")
fig.update_yaxes(type="log")  # лог-шкала, чтобы видеть маленькие p-values
fig.show()

In [16]:
def simulate_pvalues_compare_methods(
    df, metric_col, cov1="cov_7d", cov2="cov_28d",
    sample_size=10000, n_iter=500, uplift=20.0, seed=42
):
    """
    Каждый прогон:
      - случайно выбираем 2 непересекающихся подвыборки пользователей одинакового размера
      - пилоту добавляем uplift (аддитивно на метрике)
      - считаем pvalue: plain ttest, CUPED(cov1), CUPED(cov2)
    """
    rng = np.random.default_rng(seed)
    ids = df["user_id"].to_numpy()
    n = len(ids)
    if 2 * sample_size > n:
        raise ValueError(f"Слишком большой sample_size: нужно 2*sample_size <= {n}")

    p_t, p_c1, p_c2 = [], [], []

    df_idx = df.set_index("user_id")

    for _ in range(n_iter):
        chosen = rng.choice(ids, size=2*sample_size, replace=False)
        control_ids = chosen[:sample_size]
        pilot_ids   = chosen[sample_size:]

        d_control = df_idx.loc[control_ids, [metric_col, cov1, cov2]].copy()
        d_pilot   = df_idx.loc[pilot_ids,   [metric_col, cov1, cov2]].copy()

        # добавляем эффект
        d_pilot[metric_col] = d_pilot[metric_col] + uplift

        # plain
        p_t.append(ttest_pvalue(d_control[metric_col], d_pilot[metric_col]))

        # cuped
        tmp = pd.concat([
            pd.DataFrame({"pilot": 0, metric_col: d_control[metric_col], cov1: d_control[cov1], cov2: d_control[cov2]}),
            pd.DataFrame({"pilot": 1, metric_col: d_pilot[metric_col],   cov1: d_pilot[cov1],   cov2: d_pilot[cov2]}),
        ], ignore_index=True)

        p_c1.append(cuped_pvalue(tmp, metric_col=metric_col, cov_col=cov1, group_col="pilot"))
        p_c2.append(cuped_pvalue(tmp, metric_col=metric_col, cov_col=cov2, group_col="pilot"))

    return np.array(p_t), np.array(p_c1), np.array(p_c2)

In [21]:
# baseline неделя до эксперимента
baseline_begin = pd.to_datetime("16.03.2022", dayfirst=True).date()
baseline_end   = pd.to_datetime("23.03.2022", dayfirst=True).date()

df_base_metric = build_experiment_metrics(users, sales, web_logs, baseline_begin, baseline_end)
df_base_cov = build_covariates_for_users(users, sales, web_logs, 
                                         exp_begin=pd.to_datetime("23.03.2022", dayfirst=True).date(), 
                                         exp_end=pd.to_datetime("30.03.2022", dayfirst=True).date(), cov_days=(7,28))
df_base = df_base_metric.merge(df_base_cov.drop(columns=["pilot"]), on="user_id", how="left")

# симуляция
p_t, p_c7, p_c28 = simulate_pvalues_compare_methods(
    df_base,
    metric_col="total_revenue",
    cov1="cov_7d",
    cov2="cov_28d",
    sample_size=10000,
    n_iter=500,
    uplift=15, 
    seed=7
)
def ecdf_xy(pvals):
    p = np.sort(np.asarray(pvals))
    y = np.arange(1, len(p) + 1) / len(p)
    return p, y

def plot_ecdf_compare_plotly(pvals_list, titles, title="ECDF of p-values"):
    fig = go.Figure()

    for pvals, name in zip(pvals_list, titles):
        x, y = ecdf_xy(pvals)
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name=name))

    # диагональ y=x
    fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines",
                             line=dict(dash="dash"), name="y=x"))

    fig.update_layout(
        title=title,
        xaxis_title="p-value",
        yaxis_title="P(p-value ≤ x)",
        xaxis_range=[0, 1],
        yaxis_range=[0, 1],
        height=450,
        width=750
    )
    fig.show()
    
plot_ecdf_compare_plotly(
    [p_t, p_c7, p_c28],
    ["t-test", "CUPED cov_7d", "CUPED cov_28d"],
    title="Power proxy: ECDF of p-values (baseline resampling + uplift)"
)

CUPED с ковариатой за 28 дней существенно снижает дисперсию метрики (≈21%) и заметно повышает мощность теста по сравнению с обычным t-test.

В симуляциях на baseline-периоде CUPED(28d) почти всегда приводит к отклонению нулевой гипотезы при заданном эффекте, что проявляется в резком смещении распределения p-values к нулю.

В реальном эксперименте эффект и без того был очень сильным, поэтому выигрыш CUPED не приводит к меньшему p-value в конкретной реализации, однако метод обеспечивает значимый запас чувствительности для экспериментов с более слабыми эффектами.

In [22]:
from scipy.stats import mannwhitneyu

def mw_pvalue(x_control, x_pilot, alternative="two-sided"):
    x_control = np.asarray(x_control)
    x_pilot = np.asarray(x_pilot)
    x_control = x_control[~np.isnan(x_control)]
    x_pilot = x_pilot[~np.isnan(x_pilot)]
    _, p = mannwhitneyu(x_control, x_pilot, alternative=alternative)
    return float(p)


for col in ["total_revenue", "total_revenue_web", "orders_cnt"]:
    p_mw = mw_pvalue(
        df_exp[df_exp["pilot"] == 0][col],
        df_exp[df_exp["pilot"] == 1][col]
    )
    print(f"{col}: MW pvalue = {p_mw:.4g}")


total_revenue: MW pvalue = 3.701e-18
total_revenue_web: MW pvalue = 3.701e-18
orders_cnt: MW pvalue = 1.601e-27


Результаты эксперимента устойчивы к выбору статистического критерия.
Помимо классического t-test, непараметрический тест Mann–Whitney также показывает статистически значимые различия между контрольной и тестовой группами по всем ключевым бизнес-метрикам (total_revenue, total_revenue_web, orders_cnt).

Это подтверждает, что наблюдаемый эффект не обусловлен отдельными выбросами или предположениями о нормальности распределения, а отражает системный сдвиг в поведении пользователей.

Декомпозиция результатов

In [23]:
# df_exp: user_id | pilot | total_revenue | orders_cnt | ...

df = df_exp.copy()

# Average Order Value
df["aov"] = np.where(
    df["orders_cnt"] > 0,
    df["total_revenue"] / df["orders_cnt"],
    0.0
)


In [24]:
group_stats = (
    df.groupby("pilot")
      .agg(
          users=("user_id", "count"),
          revenue=("total_revenue", "mean"),
          orders=("orders_cnt", "mean"),
          aov=("aov", "mean"),
      )
)

group_stats.index = group_stats.index.map({0: "control", 1: "pilot"})
group_stats

,users,revenue,orders,aov
pilot,,,,
control,11769,861.967882,0.720027,843.803637
pilot,11564,952.253545,0.785368,932.016171


In [25]:
ctrl = group_stats.loc["control"]
pil  = group_stats.loc["pilot"]

uplift = pd.DataFrame({
    "metric": ["revenue", "orders", "aov"],
    "control": [ctrl["revenue"], ctrl["orders"], ctrl["aov"]],
    "pilot":   [pil["revenue"],  pil["orders"],  pil["aov"]],
})

uplift["abs_diff"] = uplift["pilot"] - uplift["control"]
uplift["rel_diff_%"] = uplift["abs_diff"] / uplift["control"] * 100

uplift

,metric,control,pilot,abs_diff,rel_diff_%
0,revenue,861.967882,952.253545,90.285664,10.474365
1,orders,0.720027,0.785368,0.065341,9.074823
2,aov,843.803637,932.016171,88.212534,10.454154


In [26]:
delta_orders = pil["orders"] - ctrl["orders"]
delta_aov = pil["aov"] - ctrl["aov"]

revenue_from_orders = delta_orders * ctrl["aov"]
revenue_from_aov = pil["orders"] * delta_aov

total_delta_revenue = pil["revenue"] - ctrl["revenue"]

decomp = pd.DataFrame({
    "component": ["Orders effect", "AOV effect", "Total revenue diff"],
    "value": [revenue_from_orders, revenue_from_aov, total_delta_revenue]
})

decomp

,component,value
0,Orders effect,55.135138
1,AOV effect,69.279335
2,Total revenue diff,90.285664


In [27]:
decomp["share_%"] = decomp["value"] / total_delta_revenue * 100
decomp

,component,value,share_%
0,Orders effect,55.135138,61.067433
1,AOV effect,69.279335,76.733484
2,Total revenue diff,90.285664,100.000000


In [28]:
from scipy.stats import ttest_ind, mannwhitneyu

# t-test
p_aov_t = ttest_ind(
    df[df["pilot"] == 0]["aov"],
    df[df["pilot"] == 1]["aov"],
    equal_var=False
).pvalue

# Mann–Whitney
p_aov_mw = mannwhitneyu(
    df[df["pilot"] == 0]["aov"],
    df[df["pilot"] == 1]["aov"],
    alternative="two-sided"
).pvalue

p_aov_t, p_aov_mw

(np.float64(2.2046873414655742e-15), np.float64(1.7584805480178144e-18))

In [29]:
orders_c = ctrl["orders"]
orders_p = pil["orders"]
aov_c = ctrl["aov"]
aov_p = pil["aov"]

delta_orders = orders_p - orders_c
delta_aov = aov_p - aov_c

effect_orders = delta_orders * aov_c
effect_aov = orders_c * delta_aov
effect_interaction = delta_orders * delta_aov

total = effect_orders + effect_aov + effect_interaction

decomp_exact = pd.DataFrame({
    "component": ["Orders effect", "AOV effect", "Interaction"],
    "value": [effect_orders, effect_aov, effect_interaction],
})
decomp_exact["share_%"] = decomp_exact["value"] / total * 100
decomp_exact

,component,value,share_%
0,Orders effect,55.135138,44.315694
1,AOV effect,63.515423,51.051475
2,Interaction,5.763912,4.632831


Рост выручки обусловлен сбалансированным вкладом двух факторов: увеличения частоты заказов (≈44%) и роста среднего чека (≈51%), с дополнительным положительным interaction-эффектом (≈5%).

Это указывает на то, что эксперимент одновременно влияет на поведение пользователей и структуру заказа, не создавая негативных компромиссов между метриками.